# Anthropic Financial Analyst Agent with Real-time News Access (built with LlamaIndex)

## 1. Process <br>
this notebook is designed to retrieve financial data about a stock or company:

### Tools used:


1.   Anthropic API - Claude 3.5 Sonnett   
2.   LlamaIndex for orchestration
3.   Financial Modeling Prep API for stock information
4.   News API for news summarization



In [1]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index -q

In [2]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

## 2. Instantiate Anthropic and News API Keys

Step 1: Instantiate Baseline LLM to be used


*   Experiment 1: Use Claude Sonnet for tool use, code generation and parsing
*   Experiment 2: Use OpenAI o1 models to test reasoning capabilities



In [3]:
from google.colab import userdata

CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
OAI_API_KEY = userdata.get('OPENAI_API_KEY')
FMP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

In [4]:
openai_llm = OpenAI(model="gpt-4o-mini", api_key=OAI_API_KEY)

In [5]:
anthropic_llm = Anthropic(model="claude-3-5-sonnet-20240620", api_key=CLAUDE_API_KEY)

Step 2: Instantiate News API for Retrieval

## 3. Define Tools for LLM Use

### 3.1. Stock Price Function - get_stock_price

In [6]:
import os
import requests

# Define the function that will retrieve the financial data

def get_stock_price(symbol):
    """
    Fetch the current stock price for the given symbol, the current stock trading volume, the average 50-day, 200-day moving average price,
    Earnings per share (EPS), price/earnings (P/E) ratio, and the next earnings Announcement.
    """
    url = f"https://financialmodelingprep.com/api/v3/quote-order/{symbol}?apikey={FMP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        price = data[0]['price']
        volume = data[0]['volume']
        avgVolume = data[0]['avgVolume']
        priceAvg50 = data[0]['priceAvg50']
        priceAvg200 = data[0]['priceAvg200']
        eps = data[0]['eps']
        pe = data[0]['pe']
        earningsAnnouncement = data[0]['earningsAnnouncement']
        return {"symbol": symbol.upper(), "price": price, "volume": volume, "avgVolume": avgVolume, "priceAvg50": priceAvg50, "priceAvg200": priceAvg200, "EPS": eps, "PE": pe, "earningsAnnouncement": earningsAnnouncement}
    except (IndexError, KeyError):
        return {"error": f"Invalid symbol or data not available for {symbol}"}


## DATA FORMAT OF THE STOCK PRICE FULL DATA ENDPOINT - FREE ENDPOINT (v3)
# [
# 	{
# 		"symbol": "AAPL",
# 		"name": "Apple Inc.",
# 		"price": 145.855,
# 		"changesPercentage": 0.3751,
# 		"change": 0.545,
# 		"dayLow": 143.9,
# 		"dayHigh": 146.71,
# 		"yearHigh": 179.61,
# 		"yearLow": 124.17,
# 		"marketCap": 2307703191828,
# 		"priceAvg50": 140.8724,
# 		"priceAvg200": 147.18594,
# 		"exchange": "NASDAQ",
# 		"volume": 42609394,
# 		"avgVolume": 73638864,
# 		"open": 144.38,
# 		"previousClose": 145.31,
# 		"eps": 5.89,
# 		"pe": 24.76,
# 		"earningsAnnouncement": "2023-04-26T10:59:00.000+0000",
# 		"sharesOutstanding": 15821899776,
# 		"timestamp": 1677790784
# 	}
# ]


### 3.2. Retrieve Company Financials - get_company_financials

In [7]:
def get_company_financials(symbol):
  """
  Fetch basic financial information for the given company symbol such as the␣
  industry, the sector, the name of the company, and the market capitalization.
  """
  url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FMP_API_KEY}"
  response = requests.get(url)
  data = response.json()
  try:
    results = data[0]
    financials = {
    "symbol": results["symbol"],
    "companyName": results["companyName"],
    "marketCap": results["mktCap"],
    "industry": results["industry"],
    "sector": results["sector"],
    "website": results["website"],
    "beta":results["beta"],
    "price":results["price"],
    }
    return financials
  except (IndexError, KeyError):
    return {"error": f"Invalid symbol or data not available for {symbol}"}

## DATA FORMAT OF THE STOCK PROFILE FULL DATA ENDPOINT - FREE ENDPOINT (v3)
# [
# 	{
# 		"symbol": "AAPL",
# 		"price": 178.72,
# 		"beta": 1.286802,
# 		"volAvg": 58405568,
# 		"mktCap": 2794144143933,
# 		"lastDiv": 0.96,
# 		"range": "124.17-198.23",
# 		"changes": -0.13,
# 		"companyName": "Apple Inc.",
# 		"currency": "USD",
# 		"cik": "0000320193",
# 		"isin": "US0378331005",
# 		"cusip": "037833100",
# 		"exchange": "NASDAQ Global Select",
# 		"exchangeShortName": "NASDAQ",
# 		"industry": "Consumer Electronics",
# 		"website": "https://www.apple.com",
# 		"description": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original content; Apple Card, a co-branded credit card; and Apple Pay, a cashless payment service, as well as licenses its intellectual property. The company serves consumers, and small and mid-sized businesses; and the education, enterprise, and government markets. It distributes third-party applications for its products through the App Store. The company also sells its products through its retail and online stores, and direct sales force; and third-party cellular network carriers, wholesalers, retailers, and resellers. Apple Inc. was incorporated in 1977 and is headquartered in Cupertino, California.",
# 		"ceo": "Mr. Timothy D. Cook",
# 		"sector": "Technology",
# 		"country": "US",
# 		"fullTimeEmployees": "164000",
# 		"phone": "408 996 1010",
# 		"address": "One Apple Park Way",
# 		"city": "Cupertino",
# 		"state": "CA",
# 		"zip": "95014",
# 		"dcfDiff": 4.15176,
# 		"dcf": 150.082,
# 		"image": "https://financialmodelingprep.com/image-stock/AAPL.png",
# 		"ipoDate": "1980-12-12",
# 		"defaultImage": false,
# 		"isEtf": false,
# 		"isActivelyTrading": true,
# 		"isAdr": false,
# 		"isFund": false
# 	}
# ]


### 3.3. Retrieve Key Metrics - get_key_metrics_ttm

In [8]:

def get_key_metrics_ttm(symbol):
  """
  Get key financial metrics for a company, including revenue, net income, and price-to-earnings ratio (P/E ratio). Assess a company's financial performance and compare it to its competitors.
  """
  url = f"https://financialmodelingprep.com/api/v3/key-metrics-ttm/{symbol}?apikey={FMP_API_KEY}"
  response = requests.get(url)
  data = response.json()
  try:
    results = data[0]
    key_metrics = {
    "revenuePerShareTTM": results["revenuePerShareTTM"],
    "netIncomePerShareTTM": results["netIncomePerShareTTM"],
    "operatingCashFlowPerShareTTM": results["operatingCashFlowPerShareTTM"],
    "freeCashFlowPerShareTTM": results["freeCashFlowPerShareTTM"],
    "cashPerShareTTM": results["cashPerShareTTM"],
    "bookValuePerShareTTM": results["bookValuePerShareTTM"],
    "tangibleBookValuePerShareTTM": results["tangibleBookValuePerShareTTM"],
    "marketCapTTM": results["marketCapTTM"],
    "enterpriseValueTTM": results["enterpriseValueTTM"],
    "peRatioTTM": results["peRatioTTM"],
    "priceToSalesRatioTTM": results["priceToSalesRatioTTM"],
    "priceToFCFRatioTTM": results["pfcfRatioTTM"],
    "priceToBookRatioTTM": results["ptbRatioTTM"],
    "evToSalesTTM": results["evToSalesTTM"],
    "enterpriseValueOverEBITDATTM": results["enterpriseValueOverEBITDATTM"],
    "evToOperatingCashFlowTTM": results["evToOperatingCashFlowTTM"],
    "evToFreeCashFlowTTM": results["evToFreeCashFlowTTM"],
    "earningsYieldTTM": results["earningsYieldTTM"],
    "freeCashFlowYieldTTM": results["freeCashFlowYieldTTM"],
    "debtToEquityTTM": results["debtToEquityTTM"],
    "debtToAssetsTTM": results["debtToAssetsTTM"],
    "netDebtToEBITDATTM": results["netDebtToEBITDATTM"],
    "currentRatioTTM": results["currentRatioTTM"],
    "interestCoverageTTM": results["interestCoverageTTM"],
    "roicTTM": results["roicTTM"],
    }
    return key_metrics
  except (IndexError, KeyError):
    return {"error": f"Invalid symbol or data not available for {symbol}"}


## Format follows:
# [
# 	{
# 		"revenuePerShareTTM": 24.458048210384074,
# 		"netIncomePerShareTTM": 6.036586197112504,
# 		"operatingCashFlowPerShareTTM": 7.203132909243405,
# 		"freeCashFlowPerShareTTM": 6.433270686869992,
# 		"cashPerShareTTM": 3.980350134740222,
# 		"bookValuePerShareTTM": 3.8396918155842026,
# 		"tangibleBookValuePerShareTTM": 3.8396918155842026,
# 		"shareholdersEquityPerShareTTM": 3.8396918155842026,
# 		"interestDebtPerShareTTM": 7.200966974981038,
# 		"marketCapTTM": 2767892759466,
# 		"enterpriseValueTTM": 2848764759466,
# 		"peRatioTTM": 29.32798343618193,
# 		"priceToSalesRatioTTM": 7.209311935848182,
# 		"pocfratioTTM": 24.578319216186145,
# 		"pfcfRatioTTM": 27.40840662130769,
# 		"pbRatioTTM": 46.10810150998109,
# 		"ptbRatioTTM": 46.10810150998109,
# 		"evToSalesTTM": 7.4199528549668825,
# 		"enterpriseValueOverEBITDATTM": 23.562009507183326,
# 		"evToOperatingCashFlowTTM": 25.194254629492715,
# 		"evToFreeCashFlowTTM": 28.20922256791468,
# 		"earningsYieldTTM": 0.03409712782251166,
# 		"freeCashFlowYieldTTM": 0.0364851563177914,
# 		"debtToEquityTTM": 1.8130537213392175,
# 		"debtToAssetsTTM": 0.32617195661387666,
# 		"netDebtToEBITDATTM": 0.6688887969893719,
# 		"currentRatioTTM": 0.9815625425125837,
# 		"interestCoverageTTM": 29.863225119744545,
# 		"incomeQualityTTM": 1.1932460953989026,
# 		"dividendYieldTTM": 0.005309507577062701,
# 		"dividendYieldPercentageTTM": 0.5309507577062702,
# 		"payoutRatioTTM": 0.15797804981004643,
# 		"salesGeneralAndAdministrativeToRevenueTTM": 0,
# 		"researchAndDevelopementToRevenueTTM": 0.07649511763771283,
# 		"intangiblesToTotalAssetsTTM": 0,
# 		"capexToOperatingCashFlowTTM": -0.10687880288665629,
# 		"capexToRevenueTTM": -0.03147684622056453,
# 		"capexToDepreciationTTM": -1.030176455545137,
# 		"stockBasedCompensationToRevenueTTM": 0.027312057051620986,
# 		"grahamNumberTTM": 22.83679462709757,
# 		"roicTTM": 0.5630471938175102,
# 		"returnOnTangibleAssetsTTM": 0.2828335890257225,
# 		"grahamNetNetTTM": -11.416830608779144,
# 		"workingCapitalTTM": -2304000000,
# 		"tangibleAssetValueTTM": 60274000000,
# 		"netCurrentAssetValueTTM": -152105000000,
# 		"investedCapitalTTM": 1.8130537213392175,
# 		"averageReceivablesTTM": 37542500000,
# 		"averagePayablesTTM": 44822000000,
# 		"averageInventoryTTM": 7416500000,
# 		"daysSalesOutstandingTTM": 37.25360935371536,
# 		"daysPayablesOutstandingTTM": 78.5066807297448,
# 		"daysOfInventoryOnHandTTM": 12.357922226265103,
# 		"receivablesTurnoverTTM": 9.79770836523248,
# 		"payablesTurnoverTTM": 4.649285851945438,
# 		"inventoryTurnoverTTM": 29.535709427288804,
# 		"roeTTM": 1.649211812157629,
# 		"capexPerShareTTM": -0.769862222373413,
# 		"dividendPerShareTTM": 0.94,
# 		"debtToMarketCapTTM": 0.03948129840878771
# 	}
# ]


### 3.4. Get Financial Statements: Income Statement - get_income_statement

In [9]:
def get_income_statement(symbol):
  """
  Fetch last income statement for the given company symbol such as revenue,␣
  gross profit, net income, EBITDA, EPS.
  """
  url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={FMP_API_KEY}"
  response = requests.get(url)
  data = response.json()
  try:
    results = data[0]
    financials = {
      "date": results["date"],
      "revenue": results["revenue"],
      "gross profit": results["grossProfit"],
      "net Income": results["netIncome"],
      "ebitda": results["ebitda"],
      "EPS": results["eps"],
      "EPS diluted":results["epsdiluted"]
    }
    return data, financials
  except (IndexError, KeyError):
    return {"error": f"Could not fetch financials for symbol: {symbol}"}

## Data provided by the Income Statement endpoint follows the following fomrat:
# [
# 	{
# 		"date": "2022-09-24",
# 		"symbol": "AAPL",
# 		"reportedCurrency": "USD",
# 		"cik": "0000320193",
# 		"fillingDate": "2022-10-28",
# 		"acceptedDate": "2022-10-27 18:01:14",
# 		"calendarYear": "2022",
# 		"period": "FY",
# 		"revenue": 394328000000,
# 		"costOfRevenue": 223546000000,
# 		"grossProfit": 170782000000,
# 		"grossProfitRatio": 0.4330963056,
# 		"researchAndDevelopmentExpenses": 26251000000,
# 		"generalAndAdministrativeExpenses": 0,
# 		"sellingAndMarketingExpenses": 0,
# 		"sellingGeneralAndAdministrativeExpenses": 25094000000,
# 		"otherExpenses": -334000000,
# 		"operatingExpenses": 51345000000,
# 		"costAndExpenses": 274891000000,
# 		"interestIncome": 2825000000,
# 		"interestExpense": 2931000000,
# 		"depreciationAndAmortization": 11104000000,
# 		"ebitda": 130541000000,
# 		"ebitdaratio": 0.3310467428,
# 		"operatingIncome": 119437000000,
# 		"operatingIncomeRatio": 0.302887444,
# 		"totalOtherIncomeExpensesNet": -334000000,
# 		"incomeBeforeTax": 119103000000,
# 		"incomeBeforeTaxRatio": 0.3020404333,
# 		"incomeTaxExpense": 19300000000,
# 		"netIncome": 99803000000,
# 		"netIncomeRatio": 0.2530964071,
# 		"eps": 6.15,
# 		"epsdiluted": 6.11,
# 		"weightedAverageShsOut": 16215963000,
# 		"weightedAverageShsOutDil": 16325819000,
# 		"link": "https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/0000320193-22-000108-index.htm",
# 		"finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm"
# 	}
# ]

### 3.5. Retrieve Senate Trades - get_senate_trades

In [28]:
def get_senate_trades(symbol):
  """
  Track the trading activity of US Senators and identify potential conflicts of interest. This endpoint provides a list of all the trades that have been made by US Senators, including the date of the trade, the asset traded, the amount traded, and the price per share.
  """
  url = f"https://financialmodelingprep.com/api/v4/senate-trading?symbol={symbol}&apikey={FMP_API_KEY}"
  response = requests.get(url)
  data = response.json()
  try:
    results = data[0]
    senate_trades = {
    "symbol": results["symbol"],
    "assetName": results["assetDescription"],
    "assetType": results["assetType"],
    "firstName": results["firstName"],
    "lastName": results["lastName"],
    "office": results["office"],
    "link": results["link"],
    "date": results["dateRecieved"],
    "transactionDate": results["transactionDate"],
    "owner": results["owner"],
    "type": results["type"],
    "amount": results["amount"],
    "comment": results["comment"],
    }
    return senate_trades
  except (IndexError, KeyError):
    return {"error": f"Invalid symbol or data not available for {symbol}"}


## Format of the endpoint output:
# [
# 	{
# 		"firstName": "Daniel S",
# 		"lastName": "Sullivan",
# 		"office": "Sullivan, Dan (Senator)",
# 		"link": "https://efdsearch.senate.gov/search/view/ptr/f9252211-a695-4c37-869a-0e223fce8d88/",
# 		"dateRecieved": "2023-09-21",
# 		"transactionDate": "2023-08-22",
# 		"owner": "Joint",
# 		"assetDescription": "Apple Inc",
# 		"assetType": "Stock",
# 		"type": "Sale (Full)",
# 		"amount": "$15,001 - $50,000",
# 		"comment": "Sale part of investment strategy of 3rd party investment professional to exchange individual stocks received after death of filer’s parents for EIFs.",
# 		"symbol": "AAPL"
# 	}
# ]


### 3.6. Retrieve House of Reps Trades - get_house_trades

In [43]:
def get_house_trades(symbol):
  """
  Track the trading activity of US Senators and identify potential conflicts of interest. This endpoint provides a list of all the trades that have been made by US Senators, including the date of the trade, the asset traded, the amount traded, and the price per share.
  """
  url = f"https://financialmodelingprep.com/api/v4/senate-disclosure?symbol={symbol}&apikey={FMP_API_KEY}"
  response = requests.get(url)
  data = response.json()
  try:
    results = data[0]
    house_trades = {
    "ticker": results["ticker"],
    "assetName": results["assetDescription"],
    "representative": results["representative"],
    "district": results["district"],
    "link": results["link"],
    "date": results["disclosureDate"],
    "transactionDate": results["transactionDate"],
    "owner": results["owner"],
    "type": results["type"],
    "amount": results["amount"],
    "capitalGainsOver200USD": results["capitalGainsOver200USD"],
    }
    return house_trades
  except (IndexError, KeyError):
    return {"error": f"Invalid symbol or data not available for {symbol}"}


## Format of the endpoint output:
# [
# 	{
# 		"disclosureYear": "2023",
# 		"disclosureDate": "2023-08-23",
# 		"transactionDate": "2023-08-07",
# 		"owner": "self",
# 		"ticker": "AAPL",
# 		"assetDescription": "Apple Inc.",
# 		"type": "purchase",
# 		"amount": "$1,001 - $15,000",
# 		"representative": "Michael Patrick Guest",
# 		"district": "MS03",
# 		"link": "https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2023/20023442.pdf",
# 		"capitalGainsOver200USD": "False"
# 	}
# ]



### 3.5 Test Tools and Functions

In [44]:
get_stock_price("TSLA")

{'symbol': 'TSLA',
 'price': 219.16,
 'volume': 85123635,
 'avgVolume': 85236089,
 'priceAvg50': 225.3062,
 'priceAvg200': 202.33635,
 'EPS': 3.55,
 'PE': 61.74,
 'earningsAnnouncement': '2024-10-23T00:00:00.000+0000'}

In [45]:
get_company_financials("MPC")

{'symbol': 'MPC',
 'companyName': 'Marathon Petroleum Corporation',
 'marketCap': 54386150000,
 'industry': 'Oil & Gas Refining & Marketing',
 'sector': 'Energy',
 'website': 'https://www.marathonpetroleum.com',
 'beta': 1.358,
 'price': 162.5}

In [46]:
get_income_statement("NVDA")

([{'date': '2024-01-28',
   'symbol': 'NVDA',
   'reportedCurrency': 'USD',
   'cik': '0001045810',
   'fillingDate': '2024-02-21',
   'acceptedDate': '2024-02-21 16:36:57',
   'calendarYear': '2024',
   'period': 'FY',
   'revenue': 60922000000,
   'costOfRevenue': 16621000000,
   'grossProfit': 44301000000,
   'grossProfitRatio': 0.7271757329,
   'researchAndDevelopmentExpenses': 8675000000,
   'generalAndAdministrativeExpenses': 0,
   'sellingAndMarketingExpenses': 0,
   'sellingGeneralAndAdministrativeExpenses': 2654000000,
   'otherExpenses': 237000000,
   'operatingExpenses': 11329000000,
   'costAndExpenses': 27950000000,
   'interestIncome': 866000000,
   'interestExpense': 257000000,
   'depreciationAndAmortization': 1508000000,
   'ebitda': 35583000000,
   'ebitdaratio': 0.5840747185,
   'operatingIncome': 32972000000,
   'operatingIncomeRatio': 0.5412166377,
   'totalOtherIncomeExpensesNet': 237000000,
   'incomeBeforeTax': 33818000000,
   'incomeBeforeTaxRatio': 0.555103246

In [47]:
get_senate_trades("AMD")

{'symbol': 'AMD',
 'assetName': 'Advanced Micro Devices Inc',
 'assetType': 'Stock',
 'firstName': 'Tommy',
 'lastName': 'Tuberville',
 'office': 'Tommy Tuberville',
 'link': 'https://efdsearch.senate.gov/search/view/ptr/3dabcae6-b7fd-4dd0-9ece-e936d07def11/',
 'date': '2024-02-14',
 'transactionDate': '2024-01-19',
 'owner': 'Joint',
 'type': 'Sale',
 'amount': '$15,001 - $50,000',
 'comment': ''}

In [48]:
get_house_trades("AMD")

{'ticker': 'AMD',
 'assetName': 'Advanced Micro Devices, Inc.',
 'representative': 'Josh Gottheimer',
 'district': 'NJ05',
 'link': 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2024/20025804.pdf',
 'date': '2024-09-11',
 'transactionDate': '2024-08-01',
 'owner': 'Joint',
 'type': 'Sale (Full)',
 'amount': '$1,001 - $15,000',
 'capitalGainsOver200USD': 'False'}

In [15]:
get_key_metrics_ttm("NVDA")

{'revenuePerShareTTM': 3.918422979900724,
 'netIncomePerShareTTM': 2.15672552689397,
 'operatingCashFlowPerShareTTM': 1.9799820978110505,
 'freeCashFlowPerShareTTM': 1.90357230043128,
 'cashPerShareTTM': 1.415900398730572,
 'bookValuePerShareTTM': 2.3662218243958013,
 'tangibleBookValuePerShareTTM': 2.1394336398405076,
 'marketCapTTM': 3386857100000,
 'enterpriseValueTTM': 3388309100000,
 'peRatioTTM': 64.01834553274978,
 'priceToSalesRatioTTM': 35.167299365570514,
 'priceToFCFRatioTTM': 72.39039669986748,
 'priceToBookRatioTTM': 58.35040425056312,
 'evToSalesTTM': 35.18237615126626,
 'enterpriseValueOverEBITDATTM': 53.80488931940166,
 'evToOperatingCashFlowTTM': 69.62660488245932,
 'evToFreeCashFlowTTM': 72.42143162484504,
 'earningsYieldTTM': 0.01562052239366966,
 'freeCashFlowYieldTTM': 0.013813987014686861,
 'debtToEquityTTM': 0.14978420482487062,
 'debtToAssetsTTM': 0.10220939373672662,
 'netDebtToEBITDATTM': 0.023057134690507193,
 'currentRatioTTM': 4.268952680936359,
 'interestC

## 4. Create Agentic Flow

### 4.1 Convert Functions to Tools for LLM Use

In [33]:
stock_price_tool = FunctionTool.from_defaults(fn=get_stock_price)
company_financials_tool = FunctionTool.from_defaults(fn=get_company_financials)
key_metrics_ttm_tool = FunctionTool.from_defaults(fn=get_key_metrics_ttm)
senate_trades_tool = FunctionTool.from_defaults(fn=get_senate_trades)
tool_income_statement = FunctionTool.from_defaults(fn=get_income_statement)


### 4.2 Create an Anthropic Claude Agent

In [17]:
from llama_index.core.agent import FunctionCallingAgent

claude_agent = FunctionCallingAgent.from_tools(
    [stock_price_tool, key_metrics_ttm_tool, tool_income_statement],
    llm=anthropic_llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)

### 4.3 Create an OpenAI Agent (using o1 mini Reasoning)

In [18]:
from llama_index.core.agent import FunctionCallingAgent

oai_agent = FunctionCallingAgent.from_tools(
    [stock_price_tool, company_financials_tool, key_metrics_ttm_tool, tool_income_statement],
    llm=openai_llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)

# Chat with an Agent

In [19]:
query= "Give me the current price of MPC"
response = claude_agent.chat(query)
print(str(response))

> Running step cb304ba8-585e-4bff-a9aa-887ff01810f2. Step input: Give me the current price of MPC
Added user message to memory: Give me the current price of MPC
=== LLM Response ===
Certainly! I'll fetch the current stock price for MPC (Marathon Petroleum Corporation) using the get_stock_price function. This will provide us with the current price and some additional useful information about the stock.
=== Calling Function ===
Calling function: get_stock_price with args: {"symbol": "MPC"}
=== Function Output ===
{'symbol': 'MPC', 'price': 162.5, 'volume': 1362630, 'avgVolume': 2703448, 'priceAvg50': 168.6334, 'priceAvg200': 174.57056, 'EPS': 19.07, 'PE': 8.52, 'earningsAnnouncement': '2024-11-05T13:30:00.000+0000'}
> Running step 702bcafb-562c-42ec-9fd4-d8d3f01ab29d. Step input: None
=== LLM Response ===
Based on the information retrieved, here's the current price and some additional details for MPC (Marathon Petroleum Corporation):

1. Current Price: $162.50
2. Trading Volume: 1,362,63

In [20]:
query= "Give me the current price of MPC"
response = oai_agent.chat(query)
print(str(response))

> Running step 15a4b98c-0730-4e71-8339-693be097838f. Step input: Give me the current price of MPC
Added user message to memory: Give me the current price of MPC
=== Calling Function ===
Calling function: get_stock_price with args: {"symbol": "MPC"}
=== Function Output ===
{'symbol': 'MPC', 'price': 162.5, 'volume': 1362630, 'avgVolume': 2703448, 'priceAvg50': 168.6334, 'priceAvg200': 174.57056, 'EPS': 19.07, 'PE': 8.52, 'earningsAnnouncement': '2024-11-05T13:30:00.000+0000'}
> Running step 78e887eb-cb83-45e9-bd24-fe658d64731f. Step input: None
=== LLM Response ===
The current price of MPC (Marathon Petroleum Corporation) is $162.50. Here are some additional details:

- Trading Volume: 1,362,630
- Average Volume: 2,703,448
- 50-Day Average Price: $168.63
- 200-Day Average Price: $174.57
- Earnings Per Share (EPS): $19.07
- Price/Earnings (P/E) Ratio: 8.52
- Next Earnings Announcement: November 5, 2024, at 1:30 PM UTC.
The current price of MPC (Marathon Petroleum Corporation) is $162.50.

In [21]:
query= "Give me an analysis of ARM's financial conditions"
response = claude_agent.chat(query)
print(str(response))

> Running step 8fd98c51-5246-4ac3-b411-6573e4e93574. Step input: Give me an analysis of ARM's financial conditions
Added user message to memory: Give me an analysis of ARM's financial conditions
=== LLM Response ===
Certainly! I'll provide an analysis of ARM's (Arm Holdings plc) financial conditions using several tools to gather comprehensive information. Let's start by collecting data on the company's financials, stock price, and key metrics.
=== Calling Function ===
Calling function: get_company_financials with args: {"symbol": "ARM"}
=== Function Output ===
{'symbol': 'ARM', 'companyName': 'Arm Holdings plc American Depositary Shares', 'marketCap': 169587360000, 'industry': 'Semiconductors', 'sector': 'Technology', 'website': 'https://www.arm.com', 'beta': 0, 'price': 161.82}
=== Calling Function ===
Calling function: get_stock_price with args: {"symbol": "ARM"}
=== Function Output ===
{'symbol': 'ARM', 'price': 161.82, 'volume': 8557813, 'avgVolume': 7868101, 'priceAvg50': 132.5896

In [22]:
query= "Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of MPLX"
response = claude_agent.chat(query)
print(str(response))

> Running step 44538dcb-d7f3-4e2d-92e0-3e941a7b6768. Step input: Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of MPLX
Added user message to memory: Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of MPLX
=== LLM Response ===
Certainly! I'll analyze MPLX from a value investor's perspective, focusing on the company's financial health, stability, and long-term prospects. Let's start by gathering the necessary financial information.
=== Calling Function ===
Calling function: get_company_financials with args: {"symbol": "MPLX"}
=== Function Output ===
{'symbol': 'MPLX', 'companyName': 'MPLX LP', 'marketCap': 44997372200, 'industry': 'Oil & Gas Midstream', 'sector': 'Energy', 'website': 'https://www.mplx.com', 'beta': 1.341, 'price': 44.09}
=== Calling Function ===
Calling function: get_stock_price with args: {"symbol": "MPLX"}
=== Function Output ===
{'symbol': 'MPLX', 'price': 44.09

In [23]:
query= "Give me an analysis of BMW's parent company stock profile"
response = claude_agent.chat(query)
print(str(response))

> Running step e7f49377-178c-42d5-9523-3778fc30180e. Step input: Give me an analysis of BMW's parent company stock profile
Added user message to memory: Give me an analysis of BMW's parent company stock profile
=== LLM Response ===
Certainly! I'll analyze the stock profile of BMW's parent company, which is Bayerische Motoren Werke AG, commonly known as BMW. The stock is traded under the ticker symbol "BMW" on the Frankfurt Stock Exchange. Let's gather the relevant information and provide an analysis.
=== Calling Function ===
Calling function: get_company_financials with args: {"symbol": "BMW.DE"}
=== Function Output ===
{'symbol': 'BMW.DE', 'companyName': 'Bayerische Motoren Werke Aktiengesellschaft', 'marketCap': 47790822752, 'industry': 'Auto - Manufacturers', 'sector': 'Consumer Cyclical', 'website': 'https://www.bmwgroup.com', 'beta': 1.076, 'price': 76.24}
=== Calling Function ===
Calling function: get_stock_price with args: {"symbol": "BMW.DE"}
=== Function Output ===
{'symbol': 

In [24]:
query= "Give me an analysis of BMW's parent company stock profile"
response = oai_agent.chat(query)
print(str(response))

> Running step 2e21a56a-959c-49c4-b98e-be85455dadb5. Step input: Give me an analysis of BMW's parent company stock profile
Added user message to memory: Give me an analysis of BMW's parent company stock profile
=== Calling Function ===
Calling function: get_stock_price with args: {"symbol": "BMW.DE"}
=== Function Output ===
{'symbol': 'BMW.DE', 'price': 76.24, 'volume': 652628, 'avgVolume': 1120167, 'priceAvg50': 78.7092, 'priceAvg200': 93.49125, 'EPS': 16.54, 'PE': 4.61, 'earningsAnnouncement': '2024-11-06T06:30:00.000+0000'}
=== Calling Function ===
Calling function: get_company_financials with args: {"symbol": "BMW.DE"}
=== Function Output ===
{'symbol': 'BMW.DE', 'companyName': 'Bayerische Motoren Werke Aktiengesellschaft', 'marketCap': 47790822752, 'industry': 'Auto - Manufacturers', 'sector': 'Consumer Cyclical', 'website': 'https://www.bmwgroup.com', 'beta': 1.076, 'price': 76.24}
=== Calling Function ===
Calling function: get_key_metrics_ttm with args: {"symbol": "BMW.DE"}
=== 

In [25]:
query= "Screen for 30 stocks above $1 billion in market cap that would fit the Magic Formula Investing framework"
response = claude_agent.chat(query)
print(str(response))

> Running step 07975ca6-974b-4793-80ef-f79f312297ee. Step input: Screen for 30 stocks above $1 billion in market cap that would fit the Magic Formula Investing framework
Added user message to memory: Screen for 30 stocks above $1 billion in market cap that would fit the Magic Formula Investing framework
=== LLM Response ===
To screen for 30 stocks that would fit the Magic Formula Investing framework, we need to focus on companies with high earnings yield and high return on capital. The Magic Formula, developed by Joel Greenblatt, typically looks for companies with:

1. Market capitalization above $1 billion
2. High earnings yield (EBIT / Enterprise Value)
3. High return on capital (EBIT / (Net Working Capital + Net Fixed Assets))

Unfortunately, I don't have access to a comprehensive database of all stocks and their financial metrics to perform this screen in real-time. However, I can explain the process and provide some general guidance on how you could perform this screen:

1. Start 

## 5. Testing and Chaining Different Tools

### 5.1. Politician Tracker (Senate and House Trading Activity)

In [49]:
senate_trades_tool = FunctionTool.from_defaults(fn=get_senate_trades)
house_trades_tool = FunctionTool.from_defaults(fn=get_house_trades)

In [51]:
from llama_index.core.agent import FunctionCallingAgent

claude_politician_trades_tracker = FunctionCallingAgent.from_tools(
    [stock_price_tool, tool_income_statement, senate_trades_tool, house_trades_tool],
    llm=anthropic_llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)

In [53]:
query= "Who are the most high profile politicians trading NVDA?"
response = claude_politician_trades_tracker.chat(query)
print(str(response))

> Running step ae5bd8d7-1715-450d-bdd6-b663f485ca00. Step input: Who are the most high profile politicians trading NVDA?
Added user message to memory: Who are the most high profile politicians trading NVDA?
=== LLM Response ===
To answer your question about high-profile politicians trading NVDA (NVIDIA Corporation), we'll need to check both Senate and House trading records for this stock. Let's use the available tools to gather this information.
=== Calling Function ===
Calling function: get_senate_trades with args: {"symbol": "NVDA"}
=== Function Output ===
{'symbol': 'NVDA', 'assetName': 'NVIDIA Corporation - Common Stock', 'assetType': 'Stock', 'firstName': 'Tommy', 'lastName': 'Tuberville', 'office': 'Tommy Tuberville', 'link': 'https://efdsearch.senate.gov/search/view/ptr/dda61a64-82ed-4cc8-87e7-ef1d256c6565/', 'date': '2024-05-14', 'transactionDate': '2024-04-30', 'owner': 'Joint', 'type': 'Sale (Full)', 'amount': '$15,001 - $50,000', 'comment': '--'}
=== Calling Function ===
Cal